In [1]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import wikipedia

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

GEMINI_API_KEY = os.environ.get("GEMINI_KEY_API")

UPSTASH_VECTOR_REST_URL = os.environ.get("UPSTASH_VECTOR_REST_URL")
                                         
UPSTASH_VECTOR_REST_TOKEN = os.environ.get("UPSTASH_VECTOR_REST_TOKEN")


In [4]:
ny = wikipedia.page(title= "Owerri, Imo state") 

In [55]:
# ny.content
# wikipedia.search("streptococcus pneumonia")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY



embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [33]:
from langchain_community.vectorstores import UpstashVectorStore

store = UpstashVectorStore(
    embedding=embeddings,
    index_url = UPSTASH_VECTOR_REST_URL,
    index_token = UPSTASH_VECTOR_REST_TOKEN 
)

In [34]:
from langchain_core.documents import Document

documents = []
cities = ["Owerri, Imo state", "New York, New York City", "Boise, Idaho"]

for city in cities:
    wikipedia_page_result = wikipedia.page(title=city)
    doc = Document(
        page_content = wikipedia_page_result.content,
        metadata = {
            "source": wikipedia_page_result.url,
            "title": city  
        }
    )
    documents.append(doc)
    

In [35]:
documents[2].metadata

{'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho',
 'title': 'Boise, Idaho'}

In [36]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter.from_tiktoken_encoder(model_name = "gpt-4o", chunk_size=100, chunk_overlap=0)

In [38]:
docs = text_splitter.split_documents(documents)

In [39]:
len(docs)

327

In [40]:
insert_vectors = store.add_documents(docs)

In [51]:
result = store.similarity_search_with_score("The city named after tress", k=2)

for doc, score in result:
    print(f"{doc.metadata} - {score}")

{'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho', 'title': 'Boise, Idaho'} - 0.86025506
{'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho', 'title': 'Boise, Idaho'} - 0.85201025


In [52]:
result

[(Document(metadata={'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho', 'title': 'Boise, Idaho'}, page_content=' landmark, an oasis dominated by cottonwood trees. This led the French trappers to call the area "la rivière boisée" ("the wooded river").\n\n\n==== Pronunciation ====\nMost local and longtime residents use the pronunciation /ˈbɔɪsiː/ (BOY-see), as given on the city\'s website. The pronunciation is sometimes used as a shibboleth, as outsiders (and newcomers) tend to mispronounce the city\'s name as /ˈbɔ'),
  0.86025506),
 (Document(metadata={'source': 'https://en.wikipedia.org/wiki/Boise,_Idaho', 'title': 'Boise, Idaho'}, page_content='-speaking guide, overwhelmed by the sight of the verdant river, yelled "Les bois! Les bois!" ("The woods! The woods!")—and the name stuck.\nThe name may also derive from earlier mountain men who named the river that flows through the city. In the 1820s, French Canadian fur trappers associated with the British-owned Hudson\'s Bay Company se